## Ticket System for paying for Transaction Fees

The following code is a PoC of the ticket-system descibed by Vitalik in [this](https://vitalik.ca/general/2023/01/20/stealth.html) blog post.

Essentially, users register a ticket at the coordinator by sending them small abounts of funds together with a blinded message. The coordinator verifies that the funds arrived and then signes the (chaumian) blinded message (blinded ticket) and returns it to the user. The user can then exteact the signature and ask the coordinator to fund a certain address. The coordinator would do so without being able to establish a link between the ticket purchase and the redemption.

In the following code coordinator == searcher 

In [1]:
from web3 import Web3
from eth_account import Account
import os
from Crypto.PublicKey import RSA
from Crypto.Util.number import inverse, bytes_to_long, long_to_bytes
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256

web3 = Web3(Web3.HTTPProvider('your endpoint'))

skip_broadcasting_tx = True

## CONSTANTS

In [2]:
sender_account = Account.create(os.urandom(32))
sender_private_key = sender_account.privateKey.hex()
sender_address = sender_account.address

# Searcher's address and RSA keys
searcher_address = '0xF0109fC8DF283027b6285cc889F5aA624EaC1F55'
searcher_RSA_private_key = RSA.generate(2048)
searcher_RSA_public_key  = searcher_RSA_private_key.publickey()

print(f'Sender account\nAddress: {sender_address}\nSender private Key: {sender_private_key}\n')
print(f'Searcher address: {searcher_address}')

Sender account
Address: 0xAbCD4744528F1aD0cD13004bE4Dae5179B0353C5
Sender private Key: 0x90060b269fcf40d094705f925faffa4e6758dac75c247db3fab48c3571720882

Searcher address: 0xF0109fC8DF283027b6285cc889F5aA624EaC1F55


# Sender (wants to register ticket at searcher)

## Generate ticket

In [3]:
# Sender's unblinded ticket
unblinded_ticket = b'This is Tonis ticket #1'

# Hash the unblinded ticket
unblinded_ticket_hash = SHA256.new(unblinded_ticket).digest()

# Convert unblinded ticket hash to a number for RSA operations
unblinded_ticket_long = bytes_to_long(unblinded_ticket_hash)

## Blind ticket

In [4]:
# Choose a random blinding factor and compute the blinded ticket
blinding_factor = bytes_to_long(get_random_bytes(256)) % searcher_RSA_public_key.n
blinded_ticket = (
    unblinded_ticket_long * pow(blinding_factor, searcher_RSA_public_key.e,searcher_RSA_public_key.n)
) % searcher_RSA_public_key.n

print(f'Blinded ticket:\n{web3.toHex(blinded_ticket)}')

Blinded ticket:
0x753baaab7205193f9ed327928cd8390b6f569eabd145acdd92e57cee6661642b7801da65f046bdf9e0971dac05265ffe8eee12353da633565d791269e03ef4a6db1e155806aa597414e109cfc75951d028a594a6c092eed5ec2064ceed6dc1774c969310548f98e28b752a62b379a45a4f168d23018ceec57dad12e89de1d990ddc8435aa333f905c9acb3adea7cd983d2409a76ba02c297e10850b001595b92dc0b360d5db057d2afb58b55f10e327a65abbfd07b806cad74352c312dec6e29a5d56cabbbcb32d1798eaea876e1275388f7998b34a78bce85e735d8b2907363a363df6866faee1c71c497793ed820405a54808d8bbc48d7827126b2bdb00aa


## Send funds + blinded ticket to searcher

In [5]:
# Details of the transaction
transaction = {
    'to': searcher_address,  
    'value': web3.toWei(0.01, 'ether'),  
    'gas': 21000,
    'gasPrice': web3.toWei('50', 'gwei'),
    'nonce': web3.eth.getTransactionCount(sender_account.address),  
    'chainId': 1,
    'data': web3.toHex(blinded_ticket)
}

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, sender_private_key)

# Get the raw transaction
raw_transaction = signed_txn.rawTransaction

print(f'Signed transaction\n{raw_transaction.hex()}')

Signed transaction
0xf9016d80850ba43b740082520894f0109fc8df283027b6285cc889f5aa624eac1f55872386f26fc10000b901000753baaab7205193f9ed327928cd8390b6f569eabd145acdd92e57cee6661642b7801da65f046bdf9e0971dac05265ffe8eee12353da633565d791269e03ef4a6db1e155806aa597414e109cfc75951d028a594a6c092eed5ec2064ceed6dc1774c969310548f98e28b752a62b379a45a4f168d23018ceec57dad12e89de1d990ddc8435aa333f905c9acb3adea7cd983d2409a76ba02c297e10850b001595b92dc0b360d5db057d2afb58b55f10e327a65abbfd07b806cad74352c312dec6e29a5d56cabbbcb32d1798eaea876e1275388f7998b34a78bce85e735d8b2907363a363df6866faee1c71c497793ed820405a54808d8bbc48d7827126b2bdb00aa26a08044c6c3773ea427628329d6fc24db7fbfb20c6ee03e4b98df7c29c84e50e7d3a022d803311da5de1bbb0c69a856077f134e39b337b725a83807e0dd8321fe26c9


The signed transaction can now be broadcasted

In [6]:
# Skip this cell and assumet to have a valid tx hash
if not skip_broadcasting_tx:
    sender_tx_hash = w3.eth.sendRawTransaction(signed_transaction.rawTransaction)

# Searcher (accepts tickets)

Searcher parses the transaction, verififies the receipt of funds and retrieves the blinded ticket hash

In [7]:
# Skip this cell and assumet to have a valid blinded ticket
if not skip_broadcasting_tx:
    transaction = web3.eth.getTransaction(sender_tx_hash)

    # The 'input' field contains the ticket
    blinded_ticket_hex = transaction['input']

    # Now convert the hexadecimal string back to an integer
    blinded_ticket = int(blinded_ticket_hash_hex, 16)

## Sign blinded ticket

In [8]:
# Searcher signs the blinded message
signed_blinded_ticket = pow(blinded_ticket, searcher_RSA_private_key.d, searcher_RSA_public_key.n)
print(f'Signed blinded ticket\n{web3.toHex(signed_blinded_ticket)}')

Signed blinded ticket
0x24978793605fb40fd3931fb388998645a13327c75dac2950e5351fcb8a5abc14e14bde7444d7602d54dcf3c77109a2c52c2cd95097805e20ed7164dd53c896610aae0a8f4ddce8a08ed037291ae8c52fa2b70eb6c981721f0e39f1357072470c187cd3bdc22919e3f189b9f255d40d9de69886b78a7fafb38411290ba87ce102ee3bbbe6737f2cc2edc748ed6fd5b6d4322990be5e594236a708ef41b284d1e9515340b1f66ac4d783be3887c3976dfa0b8bf4fa891dc49087c7a66dc00569dfd5ffb93b2dfecd419af5ca3737fb2cbe598ab3af4168dfde99b4b63e100996bc922967974b298622720021de7622b829114b2268d3b9eb50f33062aee31b53ff


Searcher sends the signed blinded ticket to the sender

# Sender 

In [9]:
# Sender unblinds the signed ticket hash
signature = (signed_blinded_ticket * inverse(blinding_factor, searcher_RSA_public_key.n)) % searcher_RSA_public_key.n

# Verify the signature
assert pow(signature, searcher_RSA_public_key.e, searcher_RSA_public_key.n) == unblinded_ticket_long

print("Signature of searcher valid!\n")
print(f'Searcher signature\n{web3.toHex(signature)}')

Signature of searcher valid!

Searcher signature
0x71d4f44a56ef5bec0a774fa55dee02f4f61d980c6da65cfe63cf30b50248b5fdf23a1037eb4323643be4c12d176e1b71aaaadcb5a846b11d1cfb77d94eec7d4c0424c46b65138fff3d591484314e226955083dddf084e3a9f8682e26804c3875e621c4bf68694d02c663435207cc19c438c5cc6a820eae14b33ae5382583dc54fdc5ebd05e93be808499130d32ad1b3e956615f8f742b2b4d87442024d7d592ec2506d73f9e2fb9591b11eeb75e13cce561964be5ea0bd5ae1ff37f95558c46b46fd6be262aa0e77395c373ba9ab91bade8f52a791dc40a578063f74ee502ae69bdffedb77aeb2fa62dd9421e33958d38b0cfa52813f8f9c9fc5216439daeef6


Sender can now priovide the signature to the searcher, proving it has registered a ticket 
The searcher would include the transaction in a bundle (or, because of EIP1559, fund the unfunded account of the sender). In compensation, the searcher would invalidate the used ticket by storing the signature/blinded ticket in a db.

In [10]:
transaction = {
    'to': sender_account.address,  # This would actually be the unfunded account
    'value': web3.toWei(0.01, 'ether'),  
    'gas': 21000,
    'gasPrice': web3.toWei('50', 'gwei'),
    'nonce': web3.eth.getTransactionCount(sender_account.address),  
    'chainId': 1,
    'data': web3.toHex(signature)
}

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, sender_private_key)

# Get the raw transaction
raw_transaction = signed_txn.rawTransaction

print(f'Signed transaction\n{raw_transaction.hex()}')

Signed transaction
0xf9016d80850ba43b740082520894abcd4744528f1ad0cd13004be4dae5179b0353c5872386f26fc10000b9010071d4f44a56ef5bec0a774fa55dee02f4f61d980c6da65cfe63cf30b50248b5fdf23a1037eb4323643be4c12d176e1b71aaaadcb5a846b11d1cfb77d94eec7d4c0424c46b65138fff3d591484314e226955083dddf084e3a9f8682e26804c3875e621c4bf68694d02c663435207cc19c438c5cc6a820eae14b33ae5382583dc54fdc5ebd05e93be808499130d32ad1b3e956615f8f742b2b4d87442024d7d592ec2506d73f9e2fb9591b11eeb75e13cce561964be5ea0bd5ae1ff37f95558c46b46fd6be262aa0e77395c373ba9ab91bade8f52a791dc40a578063f74ee502ae69bdffedb77aeb2fa62dd9421e33958d38b0cfa52813f8f9c9fc5216439daeef625a038594e167e1ea419c66c7caf88ae07e3498d1cb510dcb86c0f41d53d5502b647a05db308d351b785b1a0150a9e86d9dee3bbfeae334ffa0baea309b4e302d37126


Searcher could now include the transaction (although not paying anything) and invalidate the used ticket

# Searcher (verifys tickets)

In [11]:
# Validator converts the message to a long for RSA operations
original_message_long_validator = bytes_to_long(unblinded_ticket_hash)

# Validator verifies the signature
assert pow(signature, searcher_RSA_public_key.e, searcher_RSA_public_key.n) == original_message_long_validator

print("Searcher confirms the signature is valid!")

Validator confirms the signature is valid!
